In [1]:
# using kernel python
import pandas as pd
import geopandas as gpd
from pathlib import Path
from functools import reduce
import matplotlib.pyplot as plt
import json
from shapely.geometry import mapping

In [2]:
def read_geojson(path: str) -> gpd.GeoDataFrame:
    """Read a GeoJSON into a GeoDataFrame in WGS84 (EPSG:4326)."""
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"File not found: {p.resolve()}")
    gdf = gpd.read_file(p)              # <-- read the file (don’t pass the path to GeoDataFrame)
    if gdf.crs is None or gdf.crs.to_string().upper() != "EPSG:4326":
        gdf = gdf.to_crs(4326)
    return gdf


def landuse_score(path):
    df = read_geojson(path)
    df = df[df['fclass'].isin(['meadow', 'grass', 'farmland', 'scrub', 'farmyard', 'heath'])]

    sum_cols = ['area_2', 'area', 'ratio']

    # sums per id
    sums = df.groupby('id', as_index=False)[sum_cols].mean()

    # mode fclass per id (handles ties)
    modes = (df.groupby('id')['fclass']
               .agg(lambda s: s.mode().iat[0])
               .reset_index())

    # keep first row per id for other cols, then attach sums and mode fclass
    out = (df.drop_duplicates('id', keep='first')
             .drop(columns=sum_cols + ['fclass'], errors='ignore')
             .merge(sums, on='id')
             .merge(modes, on='id'))
    
    out = out[['id','ratio']]
    out = out.rename(columns={'ratio': 'land_score'})

    return out

def centroid_score(path,score_name):
    df = read_geojson(path)
    df_best = (df.sort_values(['id', 'score'], ascending=[True, False])
             .drop_duplicates('id', keep='first'))
    df_best = df_best[['id','score']]
    df_best = df_best.rename(columns={'score': f'{score_name}_score'})
    return df_best


def zonal_score(path,score_name):
    df = read_geojson(path)
    df = df[['id', '_mean']]
    df['_mean'] = pd.to_numeric(df['_mean'], errors='coerce').fillna(0)
    df = df.rename(columns={'_mean': f'{score_name}_score'})
    return df

def fill_nulls_with_zero(df: pd.DataFrame, numeric_only: bool = False, inplace: bool = False):
    """
    Fill NaNs with 0 only in columns that contain NaNs.
    If numeric_only=True, only consider numeric columns.
    Returns (df_filled, cols_filled) when inplace=False; otherwise returns cols_filled.
    """
    work = df if inplace else df.copy()

    cols = work.columns
    if numeric_only:
        cols = work.select_dtypes(include='number').columns

    cols_with_na = cols[work[cols].isna().any()]

    if len(cols_with_na) > 0:
        work[cols_with_na] = work[cols_with_na].fillna(0)

    return (work, list(cols_with_na)) if not inplace else list(cols_with_na)

def convert_geojson(gdf, out_path):
    if gdf.crs is None or gdf.crs.to_string().upper() != "EPSG:4326":
        gdf = gdf.to_crs(4326)
    gdf.to_file(out_path, driver="GeoJSON")  # writes a GeoJSON file

In [3]:

output_folder= "../../../data/output"

## box path
box_path= f"{output_folder}/box.geojson"
##  land ratio path
land_ratio_path = f"{output_folder}/land_ratio.geojson"
## zonal statistic result path
dem_path = f"{output_folder}/dem_zonal.geojson"
dni_path = f"{output_folder}/dni_zonal.geojson"
temp_path = f"{output_folder}/temp_zonal.geojson"
pvout_path = f"{output_folder}/pvout_zonal.geojson"
## centroid path
centroid_box_path= f"{output_folder}/centroid_box.geojson"
centroid_box2dso_path = f"{output_folder}/centroid_score_box2dso.geojson"
centroid_box2plant_path = f"{output_folder}/centroid_score_box2plant.geojson"
## Output path
final_score_geojson = f"{output_folder}/final_score.geojson"
final_score_csv = f"{output_folder}/final_score.csv"

In [4]:
## main reference 
box_score = read_geojson(box_path)
## zonal statistic
dem_score = zonal_score(dem_path,'dem')
dni_score = zonal_score(dni_path,'dni')
temp_score = zonal_score(temp_path,'temp')
pvout_score = zonal_score(pvout_path,'pvout')
## centroid
box2dso_score = centroid_score(centroid_box2dso_path, 'dso')
box2pv_score = centroid_score(centroid_box2plant_path, 'pv')
##land use 
land_score = landuse_score(land_ratio_path)

/Users/sell/work/code_work/daina_project/backend/.venv_backend/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: Several features with id = 670283 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(
/Users/sell/work/code_work/daina_project/backend/.venv_backend/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: Several features with id = 670283 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(
/Users/sell/work/code_work/daina_project/backend/.venv_backend/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: Several features with id = 676608 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


In [5]:
dfs = [land_score,dem_score, dni_score,temp_score,pvout_score,box2dso_score ,box2pv_score ]  # add more here
out = reduce(lambda l, r: l.merge(r, on='id', how='left'), [box_score] + dfs)
df_score,changed = fill_nulls_with_zero(out)

In [6]:
## Save result
convert_geojson(df_score, final_score_geojson)
df_score.to_csv(final_score_csv)

### Data read

In [7]:
df_box = read_geojson(box_path)
df_land_ratio = read_geojson(land_ratio_path)
df_dem = read_geojson(dem_path)
df_dni = read_geojson(dni_path)
df_temp = read_geojson(temp_path)
df_pvout = read_geojson(pvout_path)
df_centroid_box = read_geojson(centroid_box_path)
df_centroid_box2dso = read_geojson(centroid_box2dso_path)
df_centroid_box2plant = read_geojson(centroid_box2plant_path)

/Users/sell/work/code_work/daina_project/backend/.venv_backend/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: Several features with id = 676608 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(
/Users/sell/work/code_work/daina_project/backend/.venv_backend/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: Several features with id = 670283 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(
/Users/sell/work/code_work/daina_project/backend/.venv_backend/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: Several features with id = 670283 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(
/Users/sell/work/code_work/daina_project/backend/.venv_backend/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: Several features with id = 67028

In [8]:
# df_box['region'] ='dolnoslaskie'
# df_land_ratio['region'] ='dolnoslaskie'
# df_dem['region'] ='dolnoslaskie'
# df_dni['region'] ='dolnoslaskie'
# df_temp['region'] ='dolnoslaskie'
# df_pvout['region'] ='dolnoslaskie'
# df_centroid_box['region'] ='dolnoslaskie'
# df_centroid_box2dso['region'] ='dolnoslaskie'
# df_centroid_box2plant['region'] ='dolnoslaskie'
# df_score['region'] ='dolnoslaskie'

In [9]:
def rename(df, object_name, region):
    df = df.rename(columns={'id': f'{object_name}_id'})
    df['region'] = region
    return df

In [10]:
df_box=rename(df_box, 'box', 'dolnoslaskie')
df_score=rename(df_score, 'score', 'dolnoslaskie')
df_dem=rename(df_dem, 'dem', 'dolnoslaskie')
df_dni=rename(df_dni, 'dni', 'dolnoslaskie')
df_temp=rename(df_temp, 'temp', 'dolnoslaskie')
df_pvout=rename(df_pvout, 'pvout', 'dolnoslaskie')
df_centroid_box =rename(df_centroid_box, 'centroid_box', 'dolnoslaskie')
df_centroid_box2dso =rename(df_centroid_box2dso,'box2dso', 'dolnoslaskie')
df_centroid_box2plant=rename(df_centroid_box2plant,'box2plant', 'dolnoslaskie')
df_land_ratio = rename(df_land_ratio, 'land', 'dolnoslaskie')


In [30]:
fixture = []


In [42]:
fixture = []
pk_counter = 1
for _, row in df_box.iterrows():
    geom_json = mapping(row['geometry']) if row['geometry'] is not None else None
    fixture.append({
        'model': 'data.Box',
        'pk': pk_counter,
        'fields': {
            'box_id': int(row['box_id']),
            'left': float(row['left']),
            'top': float(row['top']),
            'right': float(row['right']),
            'bottom': float(row['bottom']),
            'row_index': int(row['row_index']),
            'col_index': int(row['col_index']),
            'area': float(row['area']),
            'perimeter': float(row['perimeter']),
            'geometry': geom_json,  # must be JSON-serializable (dict/list)
            'region': str(row['region'])
        }
    })
    pk_counter += 1

json_path = '../../../data/output/fixture_box.json'
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(fixture, f, indent=4)


In [43]:
fixture = []
pk_counter = 1
for _, row in df_dem.iterrows():
    geom_json = mapping(row['geometry']) if row['geometry'] is not None else None
    fixture.append({
        'model': 'data.Dem',
        'pk': pk_counter,
        'fields': {
            'dem_id': int(row['dem_id']),
            'left': float(row['left']),
            'top': float(row['top']),
            'right': float(row['right']),
            'bottom': float(row['bottom']),
            'row_index': int(row['row_index']),
            'col_index': int(row['col_index']),
            'area': float(row['area']),
            'perimeter': float(row['perimeter']),
            '_count': row['_count'],
            '_sum': row['_sum'],
            '_mean': row['_mean'],
            'geometry': geom_json,
            'region': str(row['region'])
        }
    })
    pk_counter += 1

json_path = '../../../data/output/fixture_dem.json'
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(fixture, f, indent=4)

In [44]:
fixture = []
pk_counter = 1
for _, row in df_dni.iterrows():
    geom_json = mapping(row['geometry']) if row['geometry'] is not None else None
    fixture.append({
        'model': 'data.Dni',
        'pk': pk_counter,
        'fields': {
            'dni_id': int(row['dni_id']),
            'left': float(row['left']),
            'top': float(row['top']),
            'right': float(row['right']),
            'bottom': float(row['bottom']),
            'row_index': int(row['row_index']),
            'col_index': int(row['col_index']),
            'area': float(row['area']),
            'perimeter': float(row['perimeter']),
            '_count': row['_count'],
            '_sum': row['_sum'],
            '_mean': row['_mean'],
            'geometry': geom_json,
            'region': str(row['region'])
        }
    })
    pk_counter += 1
json_path = '../../../data/output/fixture_dni.json'
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(fixture, f, indent=4)

In [45]:
fixture = []
pk_counter = 1
for _, row in df_pvout.iterrows():
    geom_json = mapping(row['geometry']) if row['geometry'] is not None else None
    fixture.append({
        'model': 'data.Pvout',
        'pk': pk_counter,
        'fields': {
            'pvout_id': int(row['pvout_id']),
            'left': float(row['left']),
            'top': float(row['top']),
            'right': float(row['right']),
            'bottom': float(row['bottom']),
            'row_index': int(row['row_index']),
            'col_index': int(row['col_index']),
            'area': float(row['area']),
            'perimeter': float(row['perimeter']),
            '_count': row['_count'],
            '_sum': row['_sum'],
            '_mean': row['_mean'],
            'geometry': geom_json,
            'region': str(row['region'])
        }
    })
    pk_counter += 1

json_path = '../../../data/output/fixture_pvout.json'
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(fixture, f, indent=4)

In [46]:
fixture = []
pk_counter = 1
for _, row in df_temp.iterrows():
    geom_json = mapping(row['geometry']) if row['geometry'] is not None else None
    fixture.append({
        'model': 'data.Temp',
        'pk': pk_counter,
        'fields': {
            'temp_id': int(row['temp_id']),
            'left': float(row['left']),
            'top': float(row['top']),
            'right': float(row['right']),
            'bottom': float(row['bottom']),
            'row_index': int(row['row_index']),
            'col_index': int(row['col_index']),
            'area': float(row['area']),
            'perimeter': float(row['perimeter']),
            '_count': row['_count'],
            '_sum': row['_sum'],
            '_mean': row['_mean'],
            'geometry': geom_json,
            'region': str(row['region'])
        }
    })
    pk_counter += 1

json_path = '../../../data/output/fixture_tem.json'
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(fixture, f, indent=4)

In [47]:

fixture = []
pk_counter = 1

for _, row in df_centroid_box.iterrows():
    geom_json = mapping(row['geometry']) if row['geometry'] is not None else None
    fixture.append({
        'model': 'data.CentroidBox',
        'pk': pk_counter,
        'fields': {
            'centroid_box_id': int(row['centroid_box_id']),
            'left': float(row['left']),
            'top': float(row['top']),
            'right': float(row['right']),
            'bottom': float(row['bottom']),
            'row_index': int(row['row_index']),
            'col_index': int(row['col_index']),
            'area': float(row['area']),
            'perimeter': float(row['perimeter']),
            'x': float(row['x']),
            'y': float(row['y']),
            'geometry': geom_json,  # must be dict or list
            'region': str(row['region'])
        }
    })
    pk_counter += 1

json_path = '../../../data/output/fixture_centroid_box.json'
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(fixture, f, indent=4)

In [52]:
fixture = []
pk_counter = 1
for _, row in df_centroid_box2dso.iterrows():
    geom_json = mapping(row['geometry']) if row['geometry'] is not None else None
    fixture.append({
        'model': 'data.CentroidBox2Dso',
        'pk': pk_counter,
        'fields': {
            'box2dso_id': int(row['box2dso_id']),
            'left': float(row['left']),
            'top': float(row['top']),
            'right': float(row['right']),
            'bottom': float(row['bottom']),
            'row_index': int(row['row_index']),
            'col_index': int(row['col_index']),
            'area': float(row['area']),
            'perimeter': float(row['perimeter']),
            'x': float(row['x']),
            'y': float(row['y']),
            'nearest_dso_id': int(row['nearest_dso_id']),
            'nearest_dso_x': float(row['nearest_dso_x']),
            'nearest_dso_y': float(row['nearest_dso_y']),
            'distance': float(row['distance_km']),
            'score': float(row['score']),
            'geometry': geom_json,  # must be dict or list
            'region': str(row['region'])
        }
    })
    pk_counter += 1

json_path = '../../../data/output/fixture_box2dso.json'
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(fixture, f, indent=4)

In [55]:
fixture = []
pk_counter = 1

for _, row in df_centroid_box2plant.iterrows():
    geom_json = mapping(row['geometry']) if row['geometry'] is not None else None
    fixture.append({
        'model': 'data.CentroidBox2Plant',
        'pk': pk_counter,
        'fields': {
            'box2plant_id': int(row['box2plant_id']),
            'left': float(row['left']),
            'top': float(row['top']),
            'right': float(row['right']),
            'bottom': float(row['bottom']),
            'row_index': int(row['row_index']),
            'col_index': int(row['col_index']),
            'area': float(row['area']),
            'perimeter': float(row['perimeter']),
            'x': float(row['x']),
            'y': float(row['y']),
            'nearest_plant_id': int(row['nearest_plant_id']),
            'nearest_plant_x': float(row['nearest_plant_x']),
            'nearest_plant_y': float(row['nearest_plant_y']),
            'distance': float(row['distance_km']),
            'score': float(row['score']),
            'geometry': geom_json,  # must be dict or list
            'region': str(row['region'])
        }
    })
    pk_counter += 1

json_path = '../../../data/output/fixture_box2plant.json'
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(fixture, f, indent=4)

In [56]:
fixture = []
pk_counter = 1

for _, row in df_land_ratio.iterrows():
    geom_json = mapping(row['geometry']) if row['geometry'] is not None else None
    fixture.append({
        'model': 'data.LandRatio',
        'pk': pk_counter,
        'fields': {
            'land_id': int(row['land_id']),
            'left': float(row['left']),
            'top': float(row['top']),
            'right': float(row['right']),
            'bottom': float(row['bottom']),
            'row_index': int(row['row_index']),
            'col_index': int(row['col_index']),
            'area': float(row['area']),
            'perimeter': float(row['perimeter']),
            'fid': int(row['fid']),
            'osm_id': int(row['osm_id']),
            'code': int(row['code']),
            'fclass': str(row['fclass']),
            'name': str(row['name']),
            'geometry': geom_json,  # must be dict or list
            'area_2': float(row['area_2']),
            'perimeter_2': float(row['perimeter_2']),
            'ratio': float(row['ratio']),
            'region': str(row['region'])
        }
    })
    pk_counter += 1

json_path = '../../../data/output/fixture_land_ratio.json'
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(fixture, f, indent=4)

In [57]:
fixture = []
pk_counter = 1

for _, row in df_score.iterrows():
    geom_json = mapping(row['geometry']) if row['geometry'] is not None else None
    fixture.append({
        'model': 'data.FinalScore',
        'pk': pk_counter,
        'fields': {
            'score_id': int(row['score_id']),
            'left': float(row['left']),
            'top': float(row['top']),
            'right': float(row['right']),
            'bottom': float(row['bottom']),
            'row_index': int(row['row_index']),
            'col_index': int(row['col_index']),
            'area': float(row['area']),
            'perimeter': float(row['perimeter']),
            'geometry': geom_json,  # must be dict or list
            'land_score': float(row['land_score']),
            'dem_score': float(row['dem_score']),
            'dni_score': float(row['dni_score']),
            'temp_score': float(row['temp_score']),
            'pvout_score': float(row['pvout_score']),
            'dso_score': float(row['dso_score']),
            'pv_score': float(row['pv_score']),
            'region': str(row['region'])

        }
    })
    pk_counter += 1

json_path = '../../../data/output/fixture_final_score.json'
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(fixture, f, indent=4)

In [41]:
json_path = '../../../data/output/fixture.json'

with open(json_path, "w", encoding="utf-8") as f:
    json.dump(fixture, f, indent=4)

In [26]:
fixture

[{'model': 'data.Box',
  'pk': 1,
  'fields': {'box_id': 676612,
   'left': 414729.5898680438,
   'top': 369375.17413494363,
   'right': 414979.5898680438,
   'bottom': 369125.17413494363,
   'row_index': 306,
   'col_index': 855,
   'area': 6656.084137585014,
   'perimeter': 410.8537858174989,
   'geometry': <MULTIPOLYGON (((17.78 51.184, 17.78 51.184, 17.78 51.184, 17.78 51.184, 17....>,
   'region': 'dolnoslaskie'}},
 {'model': 'data.Box',
  'pk': 2,
  'fields': {'box_id': 676611,
   'left': 414729.5898680438,
   'top': 369625.17413494363,
   'right': 414979.5898680438,
   'bottom': 369375.17413494363,
   'row_index': 305,
   'col_index': 855,
   'area': 28107.49915479496,
   'perimeter': 755.1391918445369,
   'geometry': <MULTIPOLYGON (((17.782 51.187, 17.782 51.187, 17.781 51.186, 17.781 51.186,...>,
   'region': 'dolnoslaskie'}},
 {'model': 'data.Box',
  'pk': 3,
  'fields': {'box_id': 676610,
   'left': 414729.5898680438,
   'top': 369875.17413494363,
   'right': 414979.58986804